## Install

Python 3 only:
```bash
pip install canton
```

## Usage

Import the essentials:

In [1]:
import tensorflow as tf
import canton as ct
import numpy as np

Define our input:

In [2]:
input_variable = tf.Variable(np.random.normal(loc=0,scale=1,size=[1,256,256,3]
    ).astype('float32'))

Then feed it through three 2-D convolutional layers, where:
- conv_0 has its own weights
- conv_1 and conv_2 share weights

In order to do this we first create 2 convolutional layers, each with its own set of weights:

In [3]:
conv = ct.Conv2D(3,16,3)
shared_conv = ct.Conv2D(16,16,3)
print(conv.weights)
print(shared_conv.weights)

[<tensorflow.python.ops.variables.Variable object at 0x0000000009215B70>, <tensorflow.python.ops.variables.Variable object at 0x0000000009215710>]
[<tensorflow.python.ops.variables.Variable object at 0x0000000009215400>, <tensorflow.python.ops.variables.Variable object at 0x000000000924F588>]


Then simply apply the second layer twice:

In [4]:
i = conv(input_variable)
i = shared_conv(i)
out = shared_conv(i)
print(out)

# define loss
loss = tf.reduce_mean(out**2.)

Tensor("add_2:0", shape=(1, 256, 256, 16), dtype=float32)


Now let's assume you only want to train the shared layer's weights (keep the first `conv` layer's weight frozen). Instead of using `tf.get_collection(some_keys_you_have_to_remember)`, or `get_layer('some_name').trainable = False`, you simply pick the weights you want to train and throw them into `optimizer.minimize()`:

In [5]:
# define optimizer
opt = tf.train.AdamOptimizer(1e-3)
# define train op
train_step = opt.minimize(loss,var_list=shared_conv.get_weights())

This seems stupid (doing more work than Keras) at first glance, but super handy if you happen to be training GANs or anything NOT for Kaggle competitions.

Now you can train it the TensorFlow way:

In [6]:
sess = ct.get_session() # just the TF Session
sess.run(tf.global_variables_initializer()) # initialize all weights
for i in range(10):
    res = sess.run([train_step,loss],feed_dict={}) # you should feed inputs if you have
    print('loss:',res[1])

loss: 0.270034
loss: 0.254609
loss: 0.240049
loss: 0.22632
loss: 0.213392
loss: 0.201226
loss: 0.18979
loss: 0.179045
loss: 0.168954
loss: 0.159481


Ok the loss is decreasing, which means the weights are getting trained. Now let's assume you like this "2Conv1Weight" idea very much, and wanna apply this layer two more times to your model:

In [7]:
out = shared_conv(out)
out = shared_conv(out)

# redefine loss
loss = tf.reduce_mean(out**2.)
# redefine train op (Note: do not redefine the optimizer, which will produce error due to variable scope clashing)
train_step = opt.minimize(loss,var_list=shared_conv.get_weights())

We don't have to reinitialize all the variables, since the previous session is still open:

In [8]:
for i in range(10):
    res = sess.run([train_step,loss],feed_dict={})
    print('loss:',res[1])

loss: 0.14364
loss: 0.13035
loss: 0.117496
loss: 0.105458
loss: 0.094434
loss: 0.0844929
loss: 0.0756286
loss: 0.0677861
loss: 0.0608834
loss: 0.0548264


As you can see from the loss value, the weights are not lost between two runs. Now let's assume you wanna save the weights to file(in numpy format) for future uses:

In [9]:
shared_conv.save_weights('shared_conv.npy')

weights (and variables) obtained.
successfully saved to shared_conv.npy


True

Now you train the model for some more steps:

In [10]:
for i in range(10):
    res = sess.run([train_step,loss],feed_dict={})
    print('loss:',res[1])

loss: 0.0495206
loss: 0.0448721
loss: 0.0407951
loss: 0.037213
loss: 0.0340581
loss: 0.0312713
loss: 0.0288014
loss: 0.0266054
loss: 0.0246466
loss: 0.0228938


Now the loss is too low, showing signs of overfitting. Assume you want to revert your weights to the last checkpoint:

In [11]:
shared_conv.load_weights('shared_conv.npy')

successfully loaded from shared_conv.npy
2 weights assigned.


True

Now train the model again. As you can see the loss values increased back to our previous checkpoint. (However the training dynamic governed by the Adam optimizer didn't change, so the results are not going to be exactly identical)

In [12]:
for i in range(10):
    res = sess.run([train_step,loss],feed_dict={})
    print('loss:',res[1])

loss: 0.0495206
loss: 0.0459745
loss: 0.0426337
loss: 0.0395109
loss: 0.0366124
loss: 0.0339363
loss: 0.0314767
loss: 0.0292234
loss: 0.0271648
loss: 0.0252875


## Concept of Cans

`Can` (`from canton import Can`) is the base class for the Conv2D layer above.

A Can is basically a container of actions and its associated weights.

When a Can is initialized, all its weight variables are created (but not initialized of course).

Every Can is callable after initialization. By calling a Can on a tensor, for example `i = shared_conv(i)`, you extend the computation graph and obtain a result tensor just like with TensorFlow, however no new weights will be created during the call. The weight is **shared** among all its calls.

As seen above, you can very easily save or restore the weights of a Can, or retrieve them as tensors. So, why not represent bigger building blocks, or even the whole network as a Can? That way we could build networks of arbitary complexity, and train them in interesting ways (like adding adversarial loss), without ever having to memorize all those variable names and scopes...

Yes, you can create Cans consisting of other Cans: that creates a Can Hierarchy.

## Can Hierarchy

Assume you came up with a new idea: Create two convolutional layer A and B, apply them one-after-another to the input N times:

- `i = B(A(i))` for N=1;

- `i = B(A(B(A(i))))` for N=2;

so why not combine A and B into one Can, and call that N times over the input? Then we only have to call `get_weights()`once to train with optimizer, call `save_weights()` once to save the parameters.

Here's the default class inheritance approach:

In [13]:
class DoubleConv(ct.Can):
    def __init__(self):
        super().__init__() # init base class
        self.convs = [ct.Conv2D(3,16,3),ct.Conv2D(16,3,3)] # define conv2d cans
        self.incan(self.convs) # add as subcans
    def __call__(self,i):
        i = self.convs[0](i)
        i = self.convs[1](i)
        return i

By calling `self.incan(cans)`, you add one or more Can(s) as the **SubCan(s)** of the Can. You can access the list of a Can's SubCans via its **subcans** property.

In [14]:
dc = DoubleConv()
print(dc.subcans)

[<canton.cans.Conv2D object at 0x000000000965CBE0>, <canton.cans.Conv2D object at 0x00000000053826D8>]


You can of course get its weights: It will traverse the hierarchy tree and collect weight tensors from its subcans.

In [15]:
print(dc.get_weights())

[<tensorflow.python.ops.variables.Variable object at 0x0000000009650D30>, <tensorflow.python.ops.variables.Variable object at 0x0000000009650F98>, <tensorflow.python.ops.variables.Variable object at 0x0000000009650B38>, <tensorflow.python.ops.variables.Variable object at 0x0000000009650F60>]


That's right, 2 convolutions needs 4 variables (2 weights and 2 biases). 

And yes, you can call it and train it, just like before:

In [16]:
i = dc(input_variable)
out = dc(i) # N=2

loss = tf.reduce_mean(out**2.)
train_step = opt.minimize(loss, var_list=dc.get_weights())

sess.run(tf.global_variables_initializer()) # init and re-init all the weights (mainly for the optimizer)
for i in range(10):
    res = sess.run([train_step,loss],feed_dict={})
    print('loss:',res[1])

loss: 0.080319
loss: 0.0681152
loss: 0.0578216
loss: 0.0492141
loss: 0.0420271
loss: 0.0360232
loss: 0.0310054
loss: 0.0268061
loss: 0.0232825
loss: 0.0203136


Again we can save and restore the Can:

In [17]:
dc.save_weights('test.npy')
dc.load_weights('test.npy')

weights (and variables) obtained.
successfully saved to test.npy
successfully loaded from test.npy
4 weights assigned.


True

## Alternative Facts

Class inheritance is boring. Is there any better ways to assemble a Can? Well you may use closure:

In [18]:
def DoubleConv2():
    can = ct.Can()
    convs = [ct.Conv2D(3,16,3),ct.Conv2D(16,3,3)]
    def call(i):
        i = convs[0](i)
        i = convs[1](i)
        return i
    can.incan(convs)
    can.set_function(call)
    return can

dc2 = DoubleConv2()
out = dc2(input_variable)

loss = tf.reduce_mean(out**2.)
train_step = opt.minimize(loss, var_list=dc2.get_weights())
sess.run(tf.global_variables_initializer()) # init and re-init all the weights (mainly for the optimizer)
for i in range(10):
    res = sess.run([train_step,loss],feed_dict={})
    print('loss:',res[1])

loss: 0.230936
loss: 0.216238
loss: 0.202303
loss: 0.189108
loss: 0.176637
loss: 0.164868
loss: 0.153776
loss: 0.143338
loss: 0.133528
loss: 0.12432


In [19]:
dc2.load_weights('test.npy')

successfully loaded from test.npy
4 weights assigned.


True

## Implement a new Can for your own need

please refer to `canton/cans.py`. Here's a simple example:

In [20]:
# you know, MLP
class Dense(ct.Can):
    def __init__(self,num_inputs,num_outputs):
        super().__init__()
        self.W = self.make_weight([num_inputs,num_outputs])
        self.b = self.make_bias([num_outputs])
    def __call__(self,i):
        W,b = self.W,self.b
        d = tf.matmul(i,W)+b
        return d